## Wheat forecast for the EU in the shadow of war:

### Wheat data:
https://www.fao.org/faostat/en/#data/QCL

countries: EU + Ukraine + russin Federation,  
Elements: Production quantity,  
Items: Wheat,  
Year: Avalible  

war flag megnézni

In [1]:
import pandas as pd 

##for ggplot
import numpy as np
from pandas.api.types import CategoricalDtype
from plotnine import *
from plotnine.data import mpg
%matplotlib inline

/Users/rajkal/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
df = pd.read_csv('wheat.csv')
df = df[["Area Code (M49)", "Area", "Year", "Value"]]
df.rename(columns={"Area Code (M49)":'Area_code'}, inplace=True)
df.head()

In [3]:
wheat_groupped = df.groupby(['Area', 'Year'])[['Value']].sum()
#type(wheat_groupped)
wheat_groupped.head(100)

Value
Area               Year         
Austria            1961   711673
                   1962   706206
                   1963   689528
                   1964   750534
                   1965   660712
...                          ...
Belgium-Luxembourg 1975   723572
                   1976   939250
                   1977   795361
                   1978  1021690
                   1979  1014501

[100 rows x 1 columns]

In [4]:
tmp = df.groupby(['Area', 'Year'], as_index=False)[['Value']].sum().pivot(index='Year', columns='Area', values='Value')
tmp['sum_production'] = tmp['Ukraine'] / tmp[tmp.columns].T.sum()
tmp['Ukraine_ratio'] = tmp['sum_production'] / tmp['Ukraine']
tmp.columns

Index(['Austria', 'Belgium', 'Belgium-Luxembourg', 'Bulgaria', 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia',
       'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal',
       'Romania', 'Russian Federation', 'Slovakia', 'Slovenia', 'Spain',
       'Sweden', 'Ukraine', 'sum_production', 'Ukraine_ratio'],
      dtype='object', name='Area')

### Weather data
https://crudata.uea.ac.uk/cru/data//hrg/cru_ts_4.06/crucy.2205251923.v4.06/

Probléma: Málta esetén furcsa adatok. Pl malta_pet -999 a teljes tábla

In [95]:
def lekerdezes(variables, countries):

    var1 = variables[0]
    cnt1 = countries[0]
    data = pd.read_csv('https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_4.06/crucy.2205251923.v4.06/countries/'+var1+'/crucy.v4.06.01.1901.2021.'+cnt1+'.'+var1+'.per', 
            delimiter=r"\s+", 
            skiprows= 3)
    nn = cnt1 + '_' + var1
    data = data.melt(id_vars=["YEAR"], 
            var_name="month", 
            value_name=nn)
    data["Date"] = data["YEAR"].astype(str) + data["month"]
    data = data[["Date", nn]]

    for varia in variables:
        for country in countries:
            if varia == "cld":
                url = 'https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_4.06/crucy.2205251923.v4.06/countries/'+varia+'/crucy.v4.06.01.1901.2021.'+country+'.'+varia+'.per'
            else:
                url = 'https://crudata.uea.ac.uk/cru/data/hrg/cru_ts_4.06/crucy.2205251923.v4.06/countries/'+varia+'/crucy.v4.06.1901.2021.'+country+'.'+varia+'.per'
            print(url)
            df = pd.read_csv(url, delimiter=r"\s+", skiprows= 3)
            nn = country + '_' + varia
            df = df.melt(id_vars=["YEAR"], 
                var_name="month", 
                value_name=nn)
            df["Date"] = df["YEAR"].astype(str) + df['month']
            data [nn] = df[[nn]]
    return data


In [ ]:
countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech_Republic', 'Denmark',
            'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland',
            'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands',
            'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden', 'Ukraine', 'Russia']
variables = ["cld", "dtr", "frs", "pet", "pre", "tmn", "tmp", "tmx", "vap", "wet"]


Weatherdata = lekerdezes(variables, countries)

In [102]:
Weatherdata.head(100000000)

,Date,Austria_cld,Belgium_cld,Bulgaria_cld,Croatia_cld,Cyprus_cld,Czech_Republic_cld,Denmark_cld,Estonia_cld,Finland_cld,...,Netherlands_wet,Poland_wet,Portugal_wet,Romania_wet,Slovakia_wet,Slovenia_wet,Spain_wet,Sweden_wet,Ukraine_wet,Russia_wet
0,1901JAN,74.3,80.6,78.3,74.1,51.6,79.5,86.6,87.7,95.9,...,16.1,14.5,15.2,14.8,14.0,9.9,15.2,14.8,12.3,15.1
1,1902JAN,79.6,89.2,76.8,76.5,49.6,85.7,89.3,87.8,95.9,...,18.1,20.4,5.6,9.8,15.8,11.8,5.0,17.8,11.2,15.8
2,1903JAN,74.4,84.9,75.9,73.9,54.9,80.3,88.3,87.7,95.9,...,17.2,18.0,13.4,10.4,14.3,10.5,14.4,16.2,10.7,15.8
3,1904JAN,79.1,85.7,77.6,76.8,54.9,85.7,89.8,87.8,95.9,...,17.9,13.3,15.1,7.6,10.7,9.7,17.3,14.8,9.7,14.8
4,1905JAN,75.3,82.5,76.0,74.2,54.8,81.2,87.2,87.6,95.9,...,16.0,17.8,11.4,10.3,13.0,8.9,13.0,14.6,11.9,15.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2052,2017ANN,64.5,73.1,56.1,53.2,33.1,72.0,74.6,74.5,83.2,...,193.5,181.3,103.1,123.0,163.8,154.4,109.5,178.8,126.7,153.4
2053,2018ANN,65.8,67.7,58.6,59.3,37.2,66.2,66.9,67.6,77.0,...,154.3,132.1,129.6,126.9,156.7,159.5,144.3,154.9,123.5,148.9
2054,2019ANN,64.9,68.4,55.8,55.0,35.7,67.2,71.6,70.5,80.2,...,197.0,148.7,102.9,119.4,160.1,168.8,117.2,179.1,117.5,153.2
2055,2020ANN,65.7,68.2,56.0,55.5,34.5,69.0,70.2,69.2,78.3,...,178.9,153.6,119.4,120.4,162.0,162.1,129.3,177.9,114.8,151.7
